In [ ]:
import os
import sys
import math
import pandas as pd
from Bio.PDB import PDBParser
from Bio.PDB.vectors import calc_dihedral

if len(sys.argv) < 2:
    print("Usage: python Angle_Calculator_simplified.py <pdb_file>")
    sys.exit(1)

input_pdb = sys.argv[1]
input_pdb_name = os.path.splitext(os.path.basename(input_pdb))[0]

output_file = "angles_output.xlsx"

# Initialize parser
parser = PDBParser(QUIET=True)
structure = parser.get_structure("Analysis", input_pdb)

def find_target_residues(structure):
    target_residues = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.get_resname() == "BLA":  # Adjust this if necessary
                    print(f"Found target residue: {residue.get_resname()} at chain {chain.id}, residue {residue.id}")
                    target_residues.append((chain.id, residue))
    return target_residues

# Function to extract atom vector safely
def get_atom_vector(residue, atom_name):
    if atom_name in residue:
        return residue[atom_name].get_vector()
    else:
        print(f"Warning: Missing atom {atom_name} in residue {residue.get_resname()}. Skipping this calculation.")
        return None

def calculate_torsion(structure, atoms):
    try:
        model = structure[0]  # First model

        def get_vector_for_atom(atom_info):
            """ Retrieves atom vector correctly for both standard residues and heteroatoms """
            chain = model[atom_info[0]] if atom_info[0] in model else None
            if not chain:
                print(f"Chain {atom_info[0]} not found.")
                return None

            # Try to find the residue directly in BioPython's chain.child_dict
            for res_id, residue in chain.child_dict.items():
                if residue.id[1] == atom_info[1]:  # Match residue number
                    atom_vector = get_atom_vector(residue, atom_info[2])
                    if atom_vector is None:
                        print(f"Atom {atom_info[2]} not found in residue {res_id}")
                    return atom_vector

            print(f"Residue {atom_info[1]} not found in chain {atom_info[0]}")
            return None

        # Retrieve atom vectors for dihedral angle calculation
        vec1 = get_vector_for_atom(atoms[0])
        vec2 = get_vector_for_atom(atoms[1])
        vec3 = get_vector_for_atom(atoms[2])
        vec4 = get_vector_for_atom(atoms[3])

        # If any atom is missing, skip calculation
        if None in [vec1, vec2, vec3, vec4]:
            print("Skipping torsion calculation due to missing atoms.")
            return None  

        # Calculate torsion angle
        torsion_angle = math.degrees(calc_dihedral(vec1, vec2, vec3, vec4))
        return round(torsion_angle, 3)

    except KeyError as e:
        print(f"Error calculating torsion angle: {e}")
        return None
    
# Prepare DataFrame
if os.path.exists(output_file):
    angles_df = pd.read_excel(output_file)
else:
    angles_df = pd.DataFrame(columns=["File", "Chain_ID", "Residue_Number", "Angle1", "Angle2", "Angle3"])

# Find target instances
target_residues = find_target_residues(structure)

# Process each target residue
for chain_id, residue in target_residues:
    res_num = residue.id[1]

    # Define three torsion angles (modify atom selection as needed)
    torsion_set1 = [(chain_id, res_num, "C1B"), (chain_id, res_num, "CHB"), (chain_id, res_num, "C4A"), (chain_id, res_num, "NA")]
    torsion_set2 = [(chain_id, res_num, "C1A"), (chain_id, res_num, "CHA"), (chain_id, res_num, "C4D"), (chain_id, res_num, "C3D")]
    torsion_set3 = [(chain_id, res_num, "C2D"), (chain_id, res_num, "C1D"), (chain_id, res_num, "CHD"), (chain_id, res_num, "C4C")]
    
    
    #torsion_set1 = [(chain_id, res_num, "C24"), (chain_id, res_num, "C21"), (chain_id, res_num, "C12"), (chain_id, res_num, "N2")]
    #torsion_set2 = [(chain_id, res_num, "C6"), (chain_id, res_num, "C7"), (chain_id, res_num, "C2"), (chain_id, res_num, "C1")]
    #torsion_set3 = [(chain_id, res_num, "C3"), (chain_id, res_num, "C4"), (chain_id, res_num, "C15"), (chain_id, res_num, "C11")]

    # Compute angles
    angle1 = calculate_torsion(structure, torsion_set1)
    angle2 = calculate_torsion(structure, torsion_set2)
    angle3 = calculate_torsion(structure, torsion_set3)

    # Append data
    new_row = pd.DataFrame({
        "File": [input_pdb_name],
        "Chain_ID": [chain_id],
        "Residue_Number": [res_num],
        "Angle1": [angle1],
        "Angle2": [angle2],
        "Angle3": [angle3]
    })
    angles_df = pd.concat([angles_df, new_row], ignore_index=True)

# Save results
angles_df.to_excel(output_file, index=False)
print(f"Angles data saved to {output_file}")